# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 23, 15, 10

while True:
    dt = datetime.now()
    if dt.month >= month and dt.day >= day:
        if dt.hour > hour or (dt.hour == hour and dt.minute > minute):
            break
    sleep(60)

# Parameter optimization

In [2]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = False

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [20, 25, 30, 35]},
              'STOCH': {'fastk_period': [3, 5, 7, 9], 'slowk_period': [2, 3, 4, 5],
                        'slowd_period': [2, 3, 5, 7], 'low_bound': [10, 15, 20, 25]}}
# optim_dict = {'RSI': {'timeperiod': [16], 'low_bound': [30]},
#                   'STOCH': {'fastk_period': [5], 'slowk_period': [4],
#                             'slowd_period': [2], 'low_bound': [30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['indicator_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat = stat.sort_values(['indicator_rank', 'pct_right_forecast_avg'], ascending=False)
stat.head(20)

Number of combinations is 3072


  0%|          | 0/3072 [00:00<?, ?it/s]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,indicator_rank
1091,STOCH_RSI,14,20,5,2,2,25,100.00,82.61,78.26,86.96,82.61,82.61,82.61,73.91,73.91,86.96,86.96,78.26,82.61,82.61,78.26,78.26,78.26,73.91,73.91,73.91,73.91,73.91,78.26,69.57,-0.94,-0.73,-0.95,-0.78,-0.95,-1.09,-0.92,-1.09,-1.18,-1.56,-1.69,-1.42,-1.95,-1.95,-1.95,-2.12,-2.19,-2.18,-2.30,-1.98,-1.89,-2.35,-1.62,-1.52,23,79.710000,223.330000
1809,STOCH_RSI,14,35,3,3,2,15,87.50,85.00,85.00,77.50,75.00,75.00,77.50,77.50,72.50,80.00,75.00,80.00,70.00,67.50,67.50,65.00,72.50,75.00,72.50,72.50,72.50,72.50,75.00,67.50,-0.25,-0.43,-0.49,-0.39,-0.32,-0.35,-0.50,-0.55,-0.69,-0.61,-0.63,-0.55,-0.46,-0.64,-0.63,-0.61,-0.65,-0.57,-0.58,-0.66,-0.59,-0.59,-0.71,-0.87,40,74.895833,195.833333
1235,STOCH_RSI,14,20,9,3,2,25,100.00,94.74,94.74,73.68,68.42,63.16,68.42,68.42,73.68,84.21,78.95,78.95,78.95,73.68,73.68,78.95,84.21,78.95,78.95,78.95,78.95,78.95,84.21,68.42,-0.71,-0.82,-0.65,-0.31,-0.56,-0.75,-0.92,-0.85,-0.73,-1.03,-1.69,-1.42,-1.98,-1.51,-1.45,-1.78,-2.09,-2.18,-2.41,-1.98,-1.89,-2.35,-1.86,-1.74,19,78.509167,161.674167
2259,STOCH_RSI,16,20,9,3,2,25,100.00,100.00,100.00,71.43,71.43,71.43,71.43,64.29,71.43,85.71,78.57,85.71,78.57,71.43,71.43,85.71,85.71,85.71,85.71,85.71,85.71,85.71,85.71,71.43,-0.65,-0.87,-0.60,-0.26,-0.66,-0.87,-0.91,-0.97,-1.18,-1.29,-1.27,-1.80,-2.18,-2.13,-2.24,-2.57,-2.06,-2.28,-2.43,-2.29,-2.44,-2.67,-2.45,-2.33,14,81.248750,157.482500
2067,STOCH_RSI,16,20,3,3,2,25,100.00,100.00,100.00,88.89,88.89,77.78,77.78,77.78,77.78,88.89,88.89,88.89,88.89,88.89,88.89,77.78,77.78,77.78,77.78,88.89,77.78,88.89,88.89,88.89,-2.44,-4.89,-3.70,-3.31,-0.95,-1.09,-0.95,-1.09,-1.63,-1.63,-3.78,-3.01,-5.33,-5.33,-4.89,-4.89,-5.11,-4.79,-5.33,-4.89,-6.58,-6.74,-7.33,-6.89,9,86.112500,145.012500
1043,STOCH_RSI,14,20,3,3,2,25,100.00,92.31,92.31,84.62,92.31,84.62,76.92,84.62,76.92,84.62,84.62,76.92,92.31,84.62,84.62,69.23,69.23,61.54,61.54,76.92,69.23,76.92,84.62,76.92,-0.92,-0.82,-1.05,-1.06,-1.06,-0.87,-0.92,-0.93,-0.93,-1.56,-1.91,-0.92,-0.70,-0.95,-1.45,-1.78,-2.59,-2.18,-2.45,-2.59,-2.81,-3.00,-3.27,-3.13,13,80.770417,140.015417
1219,STOCH_RSI,14,20,9,2,2,25,100.00,82.61,82.61,82.61,73.91,73.91,73.91,73.91,69.57,73.91,69.57,73.91,82.61,78.26,73.91,73.91,73.91,69.57,69.57,73.91,73.91,73.91,78.26,69.57,-0.99,-0.75,-0.62,-0.78,-1.15,-1.09,-0.92,-0.95,-0.93,-1.12,-1.04,-1.57,-1.35,-2.32,-2.42,-2.24,-1.54,-2.18,-2.30,-2.59,-2.56,-2.47,-2.53,-2.23,23,75.905417,135.824583
1218,STOCH_RSI,14,20,9,2,2,20,100.00,81.25,81.25,87.50,81.25,81.25,81.25,75.00,75.00,75.00,75.00,75.00,81.25,81.25,81.25,75.00,75.00,68.75,68.75,68.75,68.75,68.75,75.00,68.75,-0.93,-0.72,-0.58,-0.75,-0.97,-0.97,-0.88,-0.94,-1.06,-1.09,-1.11,-1.69,-1.37,-1.63,-1.93,-2.01,-1.46,-1.69,-1.68,-1.73,-2.20,-2.10,-2.07,-2.22,16,77.083333,113.333333
1090,STOCH_RSI,14,20,5,2,2,20,100.00,75.00,68.75,81.25,81.25,81.25,81.25,68.75,68.75,87.50,87.50,81.25,81.25,81.25,81.25,75.00,75.00,68.75,68.75,68.7

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
                  'STOCH': {'fastk_period': [7], 'slowk_period': [2],
                            'slowd_period': [2], 'low_bound': [25]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)